In [6]:
# smc_experiments.ipynb

# Step 1: Import required modules
%pip install joblib
import joblib # type: ignore
import pandas as pd # type: ignore
import sys
import os

# Step 2: Add parent directory to sys.path for module imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from features.create_features import extract_features

# Step 3: Define the prediction function
def predict_market_direction(file_path):
    model_path = "model/model.joblib"
    label_encoder_path = "model/label_encoder.joblib"

    if not os.path.exists(model_path):
        print(f"Model file not found: {model_path}")
        return []
    if not os.path.exists(label_encoder_path):
        print(f"Label encoder file not found: {label_encoder_path}")
        return []

    model = joblib.load(model_path)
    label_encoder = joblib.load(label_encoder_path)

    features_df = extract_features(file_path)
    X = features_df.drop(columns=["sma_trend", "smc_trend", "combined_trend"], errors="ignore")

    predictions_encoded = model.predict(X)
    predictions = label_encoder.inverse_transform(predictions_encoded)

    probs = model.predict_proba(X)
    class_names = label_encoder.inverse_transform(model.classes_)

    results = []
    for i in range(len(predictions)):
        prob_dict = {class_names[j]: probs[i][j] for j in range(len(class_names))}
        results.append((predictions[i], prob_dict))

    return results

# Step 4: Choose a test file
test_file = "data/EURUSD_5min.csv"  # Replace with your actual test CSV file path

# Step 5: Run prediction
results = predict_market_direction(test_file)

# Step 6: Show results
for i, (label, probs) in enumerate(results):
    print(f"Row {i+1} prediction: {label}")
    print("Probabilities:")
    for cls, prob in probs.items():
        print(f"  {cls}: {prob:.2f}")
    print()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Model file not found: model/model.joblib
